# DeepLabCut Toolbox - Colab for standard (single animal) projects!

This notebook is a modified version of the demonstration notebook provided by <font color="green">DeepLabCut</font>. For the original version refer to the [GitHub page of the DeepLabCut](https://github.com/DeepLabCut/DeepLabCut).

![DeepLabCut GIF](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1619609897110-TKSTWKEM6HTGXID9D489/triMouseDLC.gif?format=2500w)


This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

### This notebook assumes you already have a project folder with labeled data (which we have saved previously on Google Drive)!

This notebook demonstrates the necessary steps to use <font color="green">DeepLabCut</font> for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the [overview & the protocol paper](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html) for detailed explanations of each step!

Nath\*, Mathis\* et al.: Using <font color="green">DeepLabCut</font> for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: [Link](https://www.nature.com/articles/s41596-019-0176-0)

------------

## **Before you start**
First, go to ``Runtime`` -> ``change runtime type`` -> select ``Python3``, and then select ``GPU``.

In [ ]:
#(this will take a few minutes to install all the dependences!)
!apt update && apt install cuda-11-8
!pip install "deeplabcut[tf]"

**Be sure to click ``RESTART RUNTIME`` if it is displayed above before moving on!**

## Link your Google Drive (with your labeled data, or the demo data):



In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define source and destination paths
source_folder = '/content/drive/MyDrive/behavior&calcium-imaging-workshop'  # Update with your shared folder path
destination_folder = '/content/behavior&calcium-imaging-workshop'  # Content folder in Colab

# Copy files from source to destination folder
shutil.copytree(source_folder, destination_folder)

print("Files copied successfully!")

YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [ ]:
# Setup your project variables:
ProjectFolderName = 'deeplabcut-network-workshop'
VideoType = 'mp4'

#don't edit these:
videofile_path = ['/content/behavior&calcium-imaging-workshop/'+ProjectFolderName+'/videos/'] # Enter the list of videos or folder to analyze.
videofile_path

In [ ]:
import deeplabcut

In [ ]:
deeplabcut.__version__

In [ ]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before:
path_config_file = '/content/behavior&calcium-imaging-workshop/'+ProjectFolderName+'/config.yaml'
path_config_file

## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
# Then, run this cell. There are many more functions you can set here, including which network to use, you can experiment with the parameters.
# Refer to the DeepLabCut User Guide for explaination.

deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

In the next cell, we'll be updating the pose configuration file, which contains all the settings necessary for training. This step ensures that we initialize the training process using weights obtained from previous training sessions.

This step is crucial because training the network to achieve optimal performance can be time-consuming, often taking several hours. Unfortunately, this duration far exceeds the time constraints of the workshop.

In [ ]:
import yaml

pose_cfg_path = '/content/behavior&calcium-imaging-workshop/deeplabcut-network-workshop/dlc-models/iteration-5/ss_vs_sscwJan30-trainset95shuffle1/train/pose_cfg.yaml'
checkpoint = '/content/behavior&calcium-imaging-workshop/deeplabcut-network-workshop/old_dlc-models/iteration-5/ss_vs_sscwJan30-trainset95shuffle2/train/snapshot-100000'

# Load the pose_cfg.yaml file
with open(pose_cfg_path, 'r') as file:
    pose_cfg_content = yaml.safe_load(file)

# Replace the value of 'init_weights' with the checkpoint path
pose_cfg_content['init_weights'] = checkpoint

# Write the modified content back to pose_cfg.yaml
with open(pose_cfg_path, 'w') as file:
    yaml.dump(pose_cfg_content, file)


## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU...
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/wiki/DOCSTRINGS#train_network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=50, maxiters=100)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations).
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit ``STOP`` you will get a KeyInterrupt ``error``! No worries! :)**

## Start evaluating:

This function assesses a trained model's performance for a specific shuffle or shuffles at a particular state or across all states on the dataset (images). The results are stored as a .csv file in a subdirectory under **evaluation-results**.

*Look through the results in the **evaluation-results** directory.* Additionally, *review the very end of the output to identify train and test errors.*

*How do you judge the model's performance?*


In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler,
#so be sure your labels are good! (And you have trained enough ;)

## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [ ]:
videofile_path = ['/content/behavior&calcium-imaging-workshop/data/behavior-data/day0.mp4', '/content/behavior&calcium-imaging-workshop/data/behavior-data/day1.mp4]
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
!pip install mpld3

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType)

You can view these graphs in Google Drive, but they are also available for visualization within the notebook. Execute the following cell and attempt to interpret the plots.

*Is the performance of the model on our data satisfactory?*

Additionally, update the plot paths to visualize the graphs from the other videos.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import mpld3

# Define paths to the images
plot_likelihood_path = '/content/behavior&calcium-imaging-workshop/data/behavior-data/day0.mp4/plot-likelihood.png'
hist_path = '/content/behavior&calcium-imaging-workshop/data/behavior-data/day0.mp4/hist.png'
plot_path = '/content/behavior&calcium-imaging-workshop/data/behavior-data/day0.mp4/plot.png'
trajectory_path = '/content/behavior&calcium-imaging-workshop/data/behavior-data/day0.mp4/trajectory.png'

# Create a figure and axis
fig, axes = plt.subplots(2, 2)

# Read and plot each image on the corresponding axis
images = [plot_likelihood_path, hist_path, plot_path, trajectory_path]
titles = ['Plot Likelihood', 'Histogram', 'Plot', 'Trajectory']

for ax, image, title in zip(axes.ravel(), images, titles):
    img = Image.open(image)
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')

# Convert the Matplotlib figure to an interactive HTML plot using mpld3
html_plot = mpld3.fig_to_html(fig)

# Display the HTML plot
display(HTML(html_plot))

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType)

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Define the path to the labeled video
labelled_video_path = '/content/behavior&calcium-imaging-workshop/data/behavior-data/day0DLC_resnet50_ss_vs_sscwJan30shuffle1_101000_labeled.mp4'

# Read the video
video_capture = cv2.VideoCapture(labelled_video_path)

# Read the first frame
ret, frame = video_capture.read()

# Convert the frame from BGR to RGB (OpenCV reads images in BGR format)
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Create a figure and axis
fig, ax = plt.subplots()

# Display the frame using matplotlib
ax.imshow(frame_rgb)
ax.axis('off')  # Hide axes

# Convert the Matplotlib figure to an interactive HTML plot using mpld3
html_plot = mpld3.fig_to_html(fig)

# Display the HTML plot
display(HTML(html_plot))
